In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [12]:
def dataUnifier(df1, df2, df3, df4, dir_output):
    df1["Filter"] = "D1"
    df1.TimeStamp = df1.TimeStamp / 10000 
    df1.TimeStamp = df1.TimeStamp.astype('float64')
    df2["Filter"] = "D2"
    df2.TimeStamp = df2.TimeStamp / 10000 
    df2.TimeStamp = df2.TimeStamp.astype('int64')
    df3["Filter"] = "D3"
    df3.TimeStamp = df3.TimeStamp / 10000 
    df3.TimeStamp = df3.TimeStamp.astype('int64')
    df3.shape
    df4["Filter"] = "D4"
    df4.TimeStamp = df4.TimeStamp / 10000 
    df4.TimeStamp = df4.TimeStamp.astype('int64')
    df4.rename({" xPos": "xPos_Mov", \
        " yPos": "yPos_Mov",
        " Zpos": "zPos_Mov", 
        " xRotation": "xRotation",
        " yRotation": "yRotation"}, axis="columns", inplace=True)

    df_all = pd.concat([df1, df2, df3, df4], ignore_index=True)
    df_all.sort_values("TimeStamp", inplace=True, ignore_index=True)
    dfproc = df_all[df_all.Filter.ne(df_all.Filter.shift(1))]
    dfproc.reset_index(inplace=True)

    index_list = []
    collect_list = []
    count = 1
    dfproc["OrderFilter"] = -1

    for i in (range(dfproc.shape[0])):
        curr_filter = dfproc.loc[i, "Filter"]
        
        if curr_filter in collect_list:
            # Break condition 
            dfproc.loc[index_list, "OrderFilter"] = count
            collect_list = [curr_filter]
            index_list = [i]
            count += 1
        else:
            collect_list.append(curr_filter)
            index_list.append(i)

    dfproc.loc[index_list, "OrderFilter"] = count
    dfproc.rename({" xPos": "xPos", " yPos": "yPos", " zPos": "zPos", \
    " rightEye": "rightEye", " leftEye": "leftEye", 
    " xRotation": "xRotation",
    " yRotation": "yRotation",
    " zRotation": "zRotation"}, axis=1, inplace=True)

    # Create empty dataframe to merge
    event_list = list(dfproc.Filter.unique())
    event_list.sort()
    max_filter = dfproc.OrderFilter.max()
    empty_dict = {"OrderFilter": list(np.repeat(list(range(1, max_filter + 1)), len(event_list))),
    "Filter": event_list * max_filter}

    df_empty = pd.DataFrame(empty_dict)

    df_merged = dfproc.merge(df_empty, how="right", on=["OrderFilter", "Filter"])
    df_merged.drop(["index", "SessionTime"], axis="columns", inplace=True)

    df_merged.Event.replace(np.nan, "", inplace=True)

    nuller_list = ['xPos', 'yPos', 'zPos', \
        'rightEye', 'leftEye', \
        "xRotation", "yRotation", "zRotation", \
        "xPos_Mov","yPos_Mov", "zPos_Mov"]

    for nuller in nuller_list:
        df_merged[nuller].replace(np.nan, "", inplace=True)

    df_merged[nuller_list] = df_merged[nuller_list].astype(str)
    df_merged.bfill(axis="rows", inplace=True)
    
    df_grouped = df_merged.sort_values(["OrderFilter", "Filter"]).groupby(["OrderFilter", "participant", "Condition", "Tiral", "Age", "Gender",]).agg({
    "TimeStamp": 'first',
    "Filter": "|".join,
    "Event": "|".join, 
    "xPos": "".join, 
    "yPos": "".join, 
    "zPos": "".join, 
    "rightEye": "".join, 
    "leftEye": "".join, 
    "xRotation": "".join,
    "yRotation": "".join,
    "zRotation": "".join,
    "xPos_Mov": "".join, 
    "yPos_Mov": "".join, 
    "zPos_Mov": "".join
     }).reset_index()
     
    df_grouped[["Event1", "Event2", "Event3", "Event4"]] = df_grouped["Event"].str.split("|", expand=True)
    df_grouped.drop(["OrderFilter", "Filter", "Event"], axis="columns", inplace=True)
    
    
    df_grouped.to_csv(dir_output)

In [13]:
dir_d1 = "../PACanalysis/analytics"
dir_d2 = "../GazeData/analytics2" 
dir_d3 = "../Pupil Data/analytics3"
dir_d4 = "../HeadMovementData/analytics4"
dir_output = "../Unified Data/analytics_combined"

In [14]:
for i in tqdm(range(1, 40)):
    try: 
        df1 = pd.read_csv(dir_d1+"_P"+str(i)+".csv")
        df2 = pd.read_csv(dir_d2+"_P"+str(i)+".csv")
        df3 = pd.read_csv(dir_d3+"_P"+str(i)+".csv")
        df4 = pd.read_csv(dir_d4+"_P"+str(i)+".csv")
        dir_output_path = dir_output+"_P"+str(i)+".csv"
        
        dataUnifier(df1, df2, df3, df4, dir_output_path)

    except FileNotFoundError as e:
        # print(e)
        print("Skipping P{}".format(i))
        # break
    except Exception as e:
        print("Error in P{}".format(i))
        print(e.__traceback__)
        

  0%|          | 0/39 [00:00<?, ?it/s]

Skipping P1


 10%|█         | 4/39 [00:17<02:05,  3.57s/it]

Skipping P3
Error in P4


 36%|███▌      | 14/39 [04:48<12:50, 30.84s/it]

Skipping P15


 41%|████      | 16/39 [05:07<08:06, 21.14s/it]

Skipping P17


100%|██████████| 39/39 [13:10<00:00, 20.28s/it]

Skipping P30
Skipping P31
Skipping P32
Skipping P33
Skipping P34
Skipping P35
Skipping P36
Skipping P37
Skipping P38
Skipping P39


### Read the dataset 

In [289]:
df1 = pd.read_csv(dir_d1)
df2 = pd.read_csv(dir_d2)
df3 = pd.read_csv(dir_d3)
df4 = pd.read_csv(dir_d4)

pd.set_option('display.float_format', lambda x: '%.3f' % x)
df1.head(10)

,TimeStamp,participant,Condition,Tiral,Age,Gender,SessionTime,Event,xPos,yPos,zPos
0,637980746289495000,P20,tut,1,25,m,300.000,Game Start,NaN,NaN,NaN
1,637980746340630000,P20,tut,1,25,m,NaN,looking at blue cube,1.390,11.954,-3.120
2,637980746340740000,P20,tut,1,25,m,NaN,looking at blue cube,1.390,11.920,-3.120
3,637980746340859000,P20,tut,1,25,m,NaN,looking at blue cube,1.390,11.887,-3.120
4,637980746340969000,P20,tut,1,25,m,NaN,looking at blue cube,1.390,11.853,-3.120
5,637980746341079000,P20,tut,1,25,m,NaN,looking at blue cube,1.390,11.820,-3.120
6,637980746341188000,P20,tut,1,25,m,NaN,looking at blue cube,1.390,11.787,-3.120
7,637980746341298000,P20,tut,1,25,m,NaN,looking at blue cube,1.390,11.753,-3.120
8,637980746341398000,P20,tut,1,25,m,NaN,looking at blue cube,1.390,11.720,-3.120
9,637980746341518000,P20,tut,1,25,m,NaN,looking at blue cube,1.390,11.687,-3.120


In [290]:
df1["Filter"] = "D1"
df1.TimeStamp = df1.TimeStamp / 10000 
df1.TimeStamp = df1.TimeStamp.astype('float64')
df1.shape

(12544, 12)

In [291]:
df2["Filter"] = "D2"
df2.TimeStamp = df2.TimeStamp / 10000 
df2.TimeStamp = df2.TimeStamp.astype('int64')
df2.shape

# df2.dtypes

(48044, 12)

In [292]:
df3["Filter"] = "D3"
df3.TimeStamp = df3.TimeStamp / 10000 
df3.TimeStamp = df3.TimeStamp.astype('int64')
df3.shape

# df3.dtypes

(78915, 12)

In [293]:
df4["Filter"] = "D4"
df4.TimeStamp = df4.TimeStamp / 10000 
df4.TimeStamp = df4.TimeStamp.astype('int64')
df4.rename({" xPos": "xPos_Mov", \
    " yPos": "yPos_Mov",
    " Zpos": "zPos_Mov", 
    " xRotation": "xRotation",
    " yRotation": "yRotation"}, axis="columns", inplace=True)
df4.shape

(66312, 15)

### Concatenate the 4 different dataframe into one single dataframe

In [297]:
df_all = pd.concat([df1, df2, df3, df4], ignore_index=True)
df_all

,TimeStamp,participant,Condition,Tiral,Age,Gender,SessionTime,Event,xPos,yPos,zPos,Filter,rightEye,leftEye,xRotation,yRotation,zRotation,xPos_Mov,yPos_Mov,zPos_Mov
0,63798074628949.508,P20,tut,1,25,m,300.000,Game Start,NaN,NaN,NaN,D1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,63798074634063.000,P20,tut,1,25,m,NaN,looking at blue cube,1.390,11.954,-3.120,D1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,63798074634074.000,P20,tut,1,25,m,NaN,looking at blue cube,1.390,11.920,-3.120,D1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,63798074634085.898,P20,tut,1,25,m,NaN,looking at blue cube,1.390,11.887,-3.120,D1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,63798074634096.898,P20,tut,1,25,m,NaN,looking at blue cube,1.390,11.853,-3.120,D1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205810,63798075711195.000,P20,co,2,25,m,NaN,HeadPos,NaN,NaN,NaN,D4,NaN,NaN,-0.154,0.492,-0.060,12.702,8.978,0.212
205811,63798075711207.000,P20,co,2,25,m,NaN,HeadPos,NaN,NaN,NaN,D4,NaN,NaN,-0.153,0.492,-0.060,12.702,8.978,0.212
205812,63798075711218.000,P20,co,2,25,m,NaN,HeadPos,NaN,NaN,NaN,D4,NaN,NaN,-0.153,0.493,-0.060,12.702,8.979,0.211
205813,63798075711229.000,P20,co,2,25,m,NaN,HeadPos,NaN,NaN,NaN,D4,NaN,NaN,-0.153,0.493,-0.060,12.702,8.979,0.211


In [298]:
df_all.sort_values("TimeStamp", inplace=True, ignore_index=True)
# df_all
df_all[df_all.Filter == "D1"]

,TimeStamp,participant,Condition,Tiral,Age,Gender,SessionTime,Event,xPos,yPos,zPos,Filter,rightEye,leftEye,xRotation,yRotation,zRotation,xPos_Mov,yPos_Mov,zPos_Mov
1986,63798074628949.508,P20,tut,1,25,m,300.000,Game Start,NaN,NaN,NaN,D1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3309,63798074634063.000,P20,tut,1,25,m,NaN,looking at blue cube,1.390,11.954,-3.120,D1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3313,63798074634074.000,P20,tut,1,25,m,NaN,looking at blue cube,1.390,11.920,-3.120,D1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3318,63798074634085.898,P20,tut,1,25,m,NaN,looking at blue cube,1.390,11.887,-3.120,D1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3322,63798074634096.898,P20,tut,1,25,m,NaN,looking at blue cube,1.390,11.853,-3.120,D1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205365,63798075714272.602,P20,co,2,25,m,NaN,looking at gold cube,1.390,5.056,0.944,D1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
205369,63798075714284.594,P20,co,2,25,m,NaN,looking at gold cube,1.390,5.023,0.944,D1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
205372,63798075714295.500,P20,co,2,25,m,NaN,looking at gold cube,1.390,4.990,0.944,D1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
205374,63798075714308.508,P20,co,2,25,m,NaN,looking at gold cube,1.390,4.956,0.944,D1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [299]:
dfproc = df_all[df_all.Filter.ne(df_all.Filter.shift(1))]
dfproc.reset_index(inplace=True)
dfproc

,index,TimeStamp,participant,Condition,Tiral,Age,Gender,SessionTime,Event,xPos,...,zPos,Filter,rightEye,leftEye,xRotation,yRotation,zRotation,xPos_Mov,yPos_Mov,zPos_Mov
0,0,63798074611661.000,P20,tut,1,25,m,NaN,RightPupil,NaN,...,NaN,D3,4.657,4.626,NaN,NaN,NaN,NaN,NaN,NaN
1,66,63798074612389.000,P20,tut,1,25,m,NaN,looking at View wall,4.970,...,-9.850,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,67,63798074612403.000,P20,tut,1,25,m,NaN,RightPupil,NaN,...,NaN,D3,4.476,4.236,NaN,NaN,NaN,NaN,NaN,NaN
3,68,63798074612403.000,P20,tut,1,25,m,NaN,looking at View wall,4.970,...,-9.850,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,69,63798074612416.000,P20,tut,1,25,m,NaN,RightPupil,NaN,...,NaN,D3,4.455,4.345,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191168,205808,63798075716704.000,P20,co,2,25,m,NaN,RightPupil,NaN,...,NaN,D3,4.026,4.045,NaN,NaN,NaN,NaN,NaN,NaN
191169,205810,63798075716716.000,P20,co,2,25,m,NaN,looking at Play wall,1.260,...,-0.303,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191170,205811,63798075716726.000,P20,co,2,25,m,NaN,RightPupil,NaN,...,NaN,D3,-1.000,-1.000,NaN,NaN,NaN,NaN,NaN,NaN
191171,205812,63798075716727.000,P20,co,2,25,m,NaN,looking at Play wall,1.260,...,-0.303,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [300]:
index_list = []
collect_list = []
count = 1
dfproc["OrderFilter"] = -1

for i in tqdm(range(dfproc.shape[0])):
    curr_filter = dfproc.loc[i, "Filter"]
    
    if curr_filter in collect_list:
        # Break condition 
        dfproc.loc[index_list, "OrderFilter"] = count
        collect_list = [curr_filter]
        index_list = [i]
        count += 1

    else:

        collect_list.append(curr_filter)
        index_list.append(i)

dfproc.loc[index_list, "OrderFilter"] = count
dfproc

<ipython-input-300-d4429a75f6e3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfproc["OrderFilter"] = -1
  0%|          | 1/191173 [00:00<7:09:19,  7.42it/s]/Users/naren/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
100%|██████████| 191173/191173 [00:33<00:00, 5696.81it/s]


,index,TimeStamp,participant,Condition,Tiral,Age,Gender,SessionTime,Event,xPos,...,Filter,rightEye,leftEye,xRotation,yRotation,zRotation,xPos_Mov,yPos_Mov,zPos_Mov,OrderFilter
0,0,63798074611661.000,P20,tut,1,25,m,NaN,RightPupil,NaN,...,D3,4.657,4.626,NaN,NaN,NaN,NaN,NaN,NaN,1
1,66,63798074612389.000,P20,tut,1,25,m,NaN,looking at View wall,4.970,...,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,67,63798074612403.000,P20,tut,1,25,m,NaN,RightPupil,NaN,...,D3,4.476,4.236,NaN,NaN,NaN,NaN,NaN,NaN,2
3,68,63798074612403.000,P20,tut,1,25,m,NaN,looking at View wall,4.970,...,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
4,69,63798074612416.000,P20,tut,1,25,m,NaN,RightPupil,NaN,...,D3,4.455,4.345,NaN,NaN,NaN,NaN,NaN,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191168,205808,63798075716704.000,P20,co,2,25,m,NaN,RightPupil,NaN,...,D3,4.026,4.045,NaN,NaN,NaN,NaN,NaN,NaN,69196
191169,205810,63798075716716.000,P20,co,2,25,m,NaN,looking at Play wall,1.260,...,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69196
191170,205811,63798075716726.000,P20,co,2,25,m,NaN,RightPupil,NaN,...,D3,-1.000,-1.000,NaN,NaN,NaN,NaN,NaN,NaN,69197
191171,205812,63798075716727.000,P20,co,2,25,m,NaN,looking at Play wall,1.260,...,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69197


In [301]:
dfproc.rename({" xPos": "xPos", " yPos": "yPos", " zPos": "zPos", \
    " rightEye": "rightEye", " leftEye": "leftEye", 
    " xRotation": "xRotation",
    " yRotation": "yRotation",
    " zRotation": "zRotation"}, axis=1, inplace=True)
dfproc

/Users/naren/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,index,TimeStamp,participant,Condition,Tiral,Age,Gender,SessionTime,Event,xPos,...,Filter,rightEye,leftEye,xRotation,yRotation,zRotation,xPos_Mov,yPos_Mov,zPos_Mov,OrderFilter
0,0,63798074611661.000,P20,tut,1,25,m,NaN,RightPupil,NaN,...,D3,4.657,4.626,NaN,NaN,NaN,NaN,NaN,NaN,1
1,66,63798074612389.000,P20,tut,1,25,m,NaN,looking at View wall,4.970,...,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,67,63798074612403.000,P20,tut,1,25,m,NaN,RightPupil,NaN,...,D3,4.476,4.236,NaN,NaN,NaN,NaN,NaN,NaN,2
3,68,63798074612403.000,P20,tut,1,25,m,NaN,looking at View wall,4.970,...,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
4,69,63798074612416.000,P20,tut,1,25,m,NaN,RightPupil,NaN,...,D3,4.455,4.345,NaN,NaN,NaN,NaN,NaN,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191168,205808,63798075716704.000,P20,co,2,25,m,NaN,RightPupil,NaN,...,D3,4.026,4.045,NaN,NaN,NaN,NaN,NaN,NaN,69196
191169,205810,63798075716716.000,P20,co,2,25,m,NaN,looking at Play wall,1.260,...,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69196
191170,205811,63798075716726.000,P20,co,2,25,m,NaN,RightPupil,NaN,...,D3,-1.000,-1.000,NaN,NaN,NaN,NaN,NaN,NaN,69197
191171,205812,63798075716727.000,P20,co,2,25,m,NaN,looking at Play wall,1.260,...,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69197


### Create Indexing dataframe with filters from 4 dataframes to merge

In [302]:
# Create empty dataframe to merge
event_list = list(dfproc.Filter.unique())
event_list.sort()
max_filter = dfproc.OrderFilter.max()
empty_dict = {"OrderFilter": list(np.repeat(list(range(1, max_filter + 1)), len(event_list))),
 "Filter": event_list * max_filter}

df_empty = pd.DataFrame(empty_dict)
df_empty

,OrderFilter,Filter
0,1,D1
1,1,D2
2,1,D3
3,1,D4
4,2,D1
...,...,...
276787,69197,D4
276788,69198,D1
276789,69198,D2
276790,69198,D3


### Merging the dataframe

In [303]:
df_merged = dfproc.merge(df_empty, how="right", on=["OrderFilter", "Filter"])
df_merged.drop(["index", "SessionTime"], axis="columns", inplace=True)

df_merged.Event.replace(np.nan, "", inplace=True)

nuller_list = ['xPos', 'yPos', 'zPos', \
    'rightEye', 'leftEye', \
    "xRotation", "yRotation", "zRotation", \
    "xPos_Mov","yPos_Mov", "zPos_Mov"]

for nuller in nuller_list:
    df_merged[nuller].replace(np.nan, "", inplace=True)

df_merged[nuller_list] = df_merged[nuller_list].astype(str)
df_merged.bfill(axis="rows", inplace=True)
df_merged


,TimeStamp,participant,Condition,Tiral,Age,Gender,Event,xPos,yPos,zPos,Filter,rightEye,leftEye,xRotation,yRotation,zRotation,xPos_Mov,yPos_Mov,zPos_Mov,OrderFilter
0,63798074612389.000,P20,tut,1.000,25.000,m,,,,,D1,,,,,,,,,1
1,63798074612389.000,P20,tut,1.000,25.000,m,looking at View wall,4.97,5.0,-9.85,D2,,,,,,,,,1
2,63798074611661.000,P20,tut,1.000,25.000,m,RightPupil,,,,D3,4.65715,4.625565,,,,,,,1
3,63798074612403.000,P20,tut,1.000,25.000,m,,,,,D4,,,,,,,,,1
4,63798074612403.000,P20,tut,1.000,25.000,m,,,,,D1,,,,,,,,,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276787,63798075716737.000,P20,co,2.000,25.000,m,,,,,D4,,,,,,,,,69197
276788,63798075716737.000,P20,co,2.000,25.000,m,,,,,D1,,,,,,,,,69198
276789,63798075716737.000,P20,co,2.000,25.000,m,,,,,D2,,,,,,,,,69198
276790,63798075716737.000,P20,co,2.000,25.000,m,RightPupil,,,,D3,-1.0,-1.0,,,,,,,69198


### Merge and Split
Merging the grouped rows of a column into one single row separated by pipe '|'. Later splitting the columns into multiple columns to get data separated. Kind of like pivoting the data but not exactly

In [304]:
df_grouped = df_merged.sort_values(["OrderFilter", "Filter"]).groupby(["OrderFilter", "participant", "Condition", "Tiral", "Age", "Gender",]).agg({
    "TimeStamp": 'first',
    "Filter": "|".join,
    "Event": "|".join, 
    "xPos": "".join, 
    "yPos": "".join, 
    "zPos": "".join, 
    "rightEye": "".join, 
    "leftEye": "".join, 
    "xRotation": "".join,
    "yRotation": "".join,
    "zRotation": "".join,
    "xPos_Mov": "".join, 
    "yPos_Mov": "".join, 
    "zPos_Mov": "".join
     }).reset_index()

In [305]:
df_grouped[["Event1", "Event2", "Event3", "Event4"]] = df_grouped["Event"].str.split("|", expand=True)
df_grouped

,OrderFilter,participant,Condition,Tiral,Age,Gender,TimeStamp,Filter,Event,xPos,...,xRotation,yRotation,zRotation,xPos_Mov,yPos_Mov,zPos_Mov,Event1,Event2,Event3,Event4
0,1,P20,tut,1.000,25.000,m,63798074612389.000,D1|D2|D3|D4,|looking at View wall|RightPupil|,4.97,...,,,,,,,,looking at View wall,RightPupil,
1,2,P20,tut,1.000,25.000,m,63798074612403.000,D1|D2|D3|D4,|looking at View wall|RightPupil|,4.97,...,,,,,,,,looking at View wall,RightPupil,
2,3,P20,tut,1.000,25.000,m,63798074612416.000,D1|D2|D3|D4,|looking at View wall|RightPupil|,4.97,...,,,,,,,,looking at View wall,RightPupil,
3,4,P20,tut,1.000,25.000,m,63798074612437.000,D1|D2|D3|D4,|looking at View wall|RightPupil|,4.97,...,,,,,,,,looking at View wall,RightPupil,
4,5,P20,tut,1.000,25.000,m,63798074612449.000,D1|D2|D3|D4,|looking at View wall|RightPupil|,4.97,...,,,,,,,,looking at View wall,RightPupil,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69198,69194,P20,co,2.000,25.000,m,63798075716693.000,D1|D2|D3|D4,|looking at Play wall|RightPupil|,1.26,...,,,,,,,,looking at Play wall,RightPupil,
69199,69195,P20,co,2.000,25.000,m,63798075716704.000,D1|D2|D3|D4,|looking at Play wall|RightPupil|,1.26,...,,,,,,,,looking at Play wall,RightPupil,
69200,69196,P20,co,2.000,25.000,m,63798075716716.000,D1|D2|D3|D4,|looking at Play wall|RightPupil|,1.26,...,,,,,,,,looking at Play wall,RightPupil,
69201,69197,P20,co,2.000,25.000,m,63798075716727.000,D1|D2|D3|D4,|looking at Play wall|RightPupil|,1.26,...,,,,,,,,looking at Play wall,RightPupil,


In [306]:
df_grouped.to_csv("P20-combined.csv")